# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio
! pip install ipywidgets


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [3]:
dataset = fo.Dataset.from_images_dir('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 87306/87306 [5.8s elapsed, 0s remaining, 15.4K samples/s]      


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5152](http://localhost:5152)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [4]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [5]:
for sample in dataset:
    object_path = f"/Users/duyennguyen/Downloads/AIC/Batch_01/objects/{sample['video']}/{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
        
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()

### Thêm thông tin CLIP embedding.

In [6]:
all_keyframe = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [7]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [8]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [9]:
embedding_dict = {}
for v in all_video:
    clip_path = f'/Users/duyennguyen/Downloads/AIC/Batch_01/clip-features-vit-b32/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [10]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


# Query

In [11]:
computed_lancedb = fob.compute_similarity(

    # default
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,         # precomputed image embeddings
    backend="sklearn",                  
    brain_key="img_sim",                


    # # lancedb
    # dataset,
    # # table_name=dataset['lancedb_index'],         # store model's name for future use
    # model="clip-vit-base32-torch",      # store model's name for future use
    # backend="lancedb",                  # sklearn, qdrant, pinecone, milvus, lancedb
    # brain_key="lancedb_index",                # img_sim, tmp, milvus_index, lancedb_index

    # # milvus
    # dataset,
    # model="clip-vit-base32-torch",      # store model's name for future use
    # # embeddings=clip_embeddings,         # precomputed image embeddings
    # backend="lancedb",                  # sklearn, qdrant, pinecone, milvus, lancedb
    # brain_key="lancedb_index",                # img_sim, tmp, milvus_index, lancedb_index
    # # uri="tcp://localhost:19530",        # milvus server uri

    # # qdrant
    # dataset,
    # model="clip-vit-base32-torch",      # store model's name for future use
    # # embeddings=clip_embeddings,         # precomputed image embeddings
    # backend="qdrant",                  # sklearn, qdrant, pinecone, milvus, lancedb
    # brain_key="qdrant_index",                # img_sim, tmp, milvus_index, lancedb_index

    # # pinecone
    # dataset,
    # model="clip-vit-base32-torch",      # store model's name for future use
    # # embeddings=clip_embeddings,         # precomputed image embeddings
    # backend="pinecone",                  # sklearn, qdrant, pinecone, milvus, lancedb
    # brain_key="pinecone_index",                # img_sim, tmp, milvus_index, lancedb_index
        
)



In [ ]:
# import os
# # from googletrans import Translator
# import pandas as pd

# # Đường dẫn thư mục A và thư mục B
# folder_path_A = '/Users/duyennguyen/Downloads/queries-p2/'
# folder_path_B = '/Users/duyennguyen/Downloads/submission/'

# # Tạo một đối tượng Translator của Google Translate
# # translator = Translator()

# # Lặp qua tất cả các tệp trong thư mục A
# for root, dirs, files in os.walk(folder_path_A):
#     for file_name in files:
#         # Tạo đường dẫn đầy đủ đến tệp trong thư mục A
#         file_path_A = os.path.join(root, file_name)

#         # Đọc nội dung tệp
#         with open(file_path_A, 'r', encoding='utf-8') as file:
#             # Kiểm tra nếu tệp là .DS_Store thì bỏ qua
#             if file_name == '.DS_Store':
#                 continue
#             query = file.read()

#         # Dịch nội dung sang tiếng Anh
#         # translation = translator.translate(query, src='auto', dest='en')
#         # translated_query = translation.text

#         # Xử lý thêm 
#         view = dataset.sort_by_similarity(
#             query, 
#             k=20,
#             brain_key="img_sim"         
#         )

#         # Tạo một DataFrame với hai cột 'Video' và 'FrameID'
#         data = pd.DataFrame(columns=['Video', 'FrameID'])

#         for sample in view:
#             # data = data.append({'Video': sample.video, 'FrameID': sample.frameid}, ignore_index=True)
#             new_data = pd.DataFrame([(sample.video, sample.frameid)], columns=['Video', 'FrameID'])
#             data = pd.concat([data, new_data], ignore_index=True)

        
#         # Tạo đường dẫn đầy đủ đến tệp trong thư mục B
#         file_path_B = os.path.join(folder_path_B, file_name)
#         # Kiểm tra xem tệp CSV đã tồn tại trong thư mục B chưa
#         if not os.path.exists(file_path_B):
#             # Nếu chưa tồn tại, tạo mới tệp CSV trước khi ghi vào
#             with open(file_path_B, 'w', encoding='utf-8') as file:
#                 pass  # Tạo tệp mới trống

#         # Ghi kết quả vào tệp CSV
#         with open(file_path_B, 'a', encoding='utf-8') as file:
#             data.to_csv(file_path_B, index=False)

#         print(f"Đã xử lý và lưu kết quả vào tệp {file_path_B}")


Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-2.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-3.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-1.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-4.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-5.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-7.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-6.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-30.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-24.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-18.txt
Đã xử lý và lưu kết quả vào tệp /Users/duyennguyen/Downloads/submission/query-p2-19.txt
Đã xử lý và lưu kết quả vào tệp /Users/

In [ ]:
# import os

# # Đường dẫn đến thư mục chứa các tệp .txt
# thu_muc = '/Users/duyennguyen/Downloads/submission/'

# # Lặp qua tất cả các tệp trong thư mục
# for ten_tep in os.listdir(thu_muc):
#     if ten_tep.endswith('.txt'):
#         duong_dan_cu = os.path.join(thu_muc, ten_tep)
#         ten_tep_moi = os.path.splitext(ten_tep)[0] + '.csv'
#         duong_dan_moi = os.path.join(thu_muc, ten_tep_moi)

#         # Đổi tên tệp từ .txt thành .csv
#         os.rename(duong_dan_cu, duong_dan_moi)

# print("Đã đổi tất cả các tệp .txt thành .csv trong thư mục.")


Đã đổi tất cả các tệp .txt thành .csv trong thư mục.


## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [12]:
# Bạn cần phải cài version umap-learn hỗ trợ.
view = fob.compute_visualization(
    dataset,
    embeddings=clip_embeddings,   # precomputed image embeddings
    brain_key="img_viz",          # img_viz, tmp, gt_viz
    method="umap",                # umap, tsne, pca
)


Generating visualization...


/Users/duyennguyen/anaconda3/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/duyennguyen/anaconda3/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/duyennguyen/anaconda3/lib/python3.10/site-packages/umap/distances.py:1086: 

UMAP( verbose=True)
Sat Oct 14 10:10:50 2023 Construct fuzzy simplicial set
Sat Oct 14 10:10:50 2023 Finding Nearest Neighbors
Sat Oct 14 10:10:50 2023 Building RP forest with 20 trees
Sat Oct 14 10:10:53 2023 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	 5  /  16
	Stopping threshold met -- exiting after 5 iterations
Sat Oct 14 10:10:59 2023 Finished Nearest Neighbor Search
Sat Oct 14 10:11:00 2023 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

Sat Oct 14 10:11:19 2023 Finished embedding


In [13]:
session = fo.launch_app(dataset, port =5151)
# session.wait()